#  Capstone Project

## Introduction/Business Problem  

### Objective:
The idea of this Data-Science project is to build a recommendation system for different types of categories like Entertainment , Fast Food , Indian Food , Sports etc.
### Target Audience:
The Business Problem is related to many tourists who travels unknown cities/countries. But while travelling, most of the times people have to search for stay, food and other activities in famous area of that City. They always wish that they get all the same required things like services, enjoyments, clubs, restaurants, hangout places etc near the area City/Country. So Is there a way we can recommend them best area in new Cities/Countries.
### Introduction / Business Problem :
So Here in this project, we are going to recommend the best areas in terms of their service, search for the potential explanation of why an area is popular, the reviews of the places in the areas, or anything else related to areas.

   #### Success criteria of the project are :
     - define common cluster/class values for similar categories.
     - deliver optimized model for these classes.
     - provide a list of similar Area within the chosen city.
     - show the recommended areas on a map.



## Data Gathering, Cleansing and Exploratory Data Analysis

### Importing libs 

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from bs4 import BeautifulSoup # library of Beautifulsoup

from folium.plugins import MarkerCluster 
import folium  # plotting library
 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import pgeocode
nomi = pgeocode.Nominatim('IN')

import matplotlib.cm as cm
import matplotlib.colors as colors

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# To Download postal codes data
import requests
from zipfile import ZipFile
from io import BytesIO
import os

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
pd.set_option('display.max_columns', None)



print('Folium installed')
print('Libraries imported.')


CLIENT_ID = 'WJALSUPQARDIIVEU4NV1RWEEFGT0DZNNX0KQTVCSX5LZNIGI' # your Foursquare ID
CLIENT_SECRET = 'RUZPK1EYFBKDLKNTDF1QKHBOMGYWG3C0JICCP0Y1C5S2LCPK' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
radius = 1000

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

filename = nomi.country+".txt"

if not os.path.isfile(filename):
    print("Downloading Postalcodes Data for {} ".format(nomi.country))
    down_url = pgeocode.DOWNLOAD_URL.replace("{country}",nomi.country)
    response = requests.get(down_url)
    z = ZipFile(BytesIO(response.content))
    z.extract(filename)

Folium installed
Libraries imported.
Your credentails:
CLIENT_ID: WJALSUPQARDIIVEU4NV1RWEEFGT0DZNNX0KQTVCSX5LZNIGI
CLIENT_SECRET:RUZPK1EYFBKDLKNTDF1QKHBOMGYWG3C0JICCP0Y1C5S2LCPK


In [3]:
india_df = pd.read_csv(filename,sep="\t",header=None,names =pgeocode.DATA_FIELDS )

ahmdbd_df = india_df[(india_df.state_name == "Gujarat")&(india_df.county_name=="Ahmedabad")]

ahmdbd_df.head()

,country code,postal_code,place_name,state_name,state_code,county_name,county_code,community_name,community_code,latitude,longitude,accuracy
17539,IN,380001,Gheekanta Road,Gujarat,9,Ahmedabad,474.0,Ahmadabad City,NaN,23.0256,72.5769,3
17540,IN,380001,Lal Darwaja,Gujarat,9,Ahmedabad,474.0,Ahmadabad City,NaN,23.0256,72.5769,4
17541,IN,380001,Manek Chowk,Gujarat,9,Ahmedabad,474.0,Ahmadabad City,NaN,23.0256,72.5769,3
17542,IN,380001,District Court (Ahmedabad),Gujarat,9,Ahmedabad,474.0,Ahmadabad City,NaN,23.0256,72.5769,3
17543,IN,380001,Gandhi Road (Ahmedabad),Gujarat,9,Ahmedabad,474.0,Ahmadabad City,NaN,23.0256,72.5769,3


In [4]:
clr_i = 0

some_map = folium.Map(location=(23.0225,72.5714) , zoom_start=9)

colors_array = cm.rainbow(np.linspace(0, 13, ahmdbd_df.shape[0]))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for city in ahmdbd_df.community_name.unique():
    city_neighbs = ahmdbd_df.loc[ahmdbd_df.community_name == city,['community_name','place_name',"postal_code",'latitude', 'longitude']]
    neighs = folium.map.FeatureGroup()
    clr = rainbow[clr_i]
    clr_i+=1
    for br,nm,pc,lat,lng in city_neighbs.values:
        folium.CircleMarker(
            [lat, lng],
            radius=3, 
            color=clr,
            fill=True,
            popup= "<br>community_name ==> {} And <br>place_name ==> {}".format(br,nm),
            fill_opacity=0.8
        ).add_to(some_map)

some_map

In [5]:
ahmd_city_df = ahmdbd_df.loc[ahmdbd_df.community_name=="Ahmadabad City",['postal_code', 'place_name', 'county_name',  'community_name', 'latitude', 'longitude', ]]
ahmd_city_df.head()

,postal_code,place_name,county_name,community_name,latitude,longitude
17539,380001,Gheekanta Road,Ahmedabad,Ahmadabad City,23.0256,72.5769
17540,380001,Lal Darwaja,Ahmedabad,Ahmadabad City,23.0256,72.5769
17541,380001,Manek Chowk,Ahmedabad,Ahmadabad City,23.0256,72.5769
17542,380001,District Court (Ahmedabad),Ahmedabad,Ahmadabad City,23.0256,72.5769
17543,380001,Gandhi Road (Ahmedabad),Ahmedabad,Ahmadabad City,23.0256,72.5769


In [6]:
ahmd_city_df.isna().sum()

postal_code       0
place_name        0
county_name       0
community_name    0
latitude          0
longitude         0
dtype: int64

In [7]:
cluster_assigned_df = ahmd_city_df.groupby("postal_code").apply(lambda x : ','.join(x["place_name"].tolist())).reset_index().rename(columns={0:"place_names"})

In [8]:
cluster_assigned_df

,postal_code,place_names
0,380001,"Gheekanta Road,Lal Darwaja,Manek Chowk,Distric..."
1,380002,"Revdibazar H.O,Railwaypura,N C Market"
2,380004,"Delhi Gate (Ahmedabad),Cantonment,Girdharnagar..."
3,380005,"Sabarmati,Kabir Chowk,O N G C"
4,380006,Ellisbridge
5,380007,"Sarkhej Road,Anandnagar (Ahmedabad),Shardanaga..."
6,380008,"Daxini Society,Khokhara Mehmadabad,L G Hospita..."
7,380009,"Gujarat University,Navrangpura H.O"
8,380013,"Naranpura Vistar,Vadaj,Nava Vadaj,Stadium Marg..."
9,380014,Navjivan


In [9]:
# getting lats and long from given method so using this file

def get_long_lat(postcode):
    """ Method takes a Series object and returns
    a list of Latitude and corresponding Longitude data,
    using the pgeocode library.
    This method also prints out the coordinate data"""
        
    location = nomi.query_postal_code(postcode)
    latitude = location.latitude
    longitude = location.longitude
    return [latitude, longitude]

In [10]:
clusters_df = cluster_assigned_df.copy()

latt_longs = clusters_df["postal_code"].apply(get_long_lat)

clusters_df.loc[:,"Latitude"] = latt_longs.apply(lambda col: col[0])
clusters_df.loc[:,"Longitude"] = latt_longs.apply(lambda col: col[1])
clusters_df.loc[:,"community_name"] = "Ahmadabad City"

# Showing first 5 rows 

clusters_df.head()

,postal_code,place_names,Latitude,Longitude,community_name
0,380001,"Gheekanta Road,Lal Darwaja,Manek Chowk,Distric...",23.0256,72.5769,Ahmadabad City
1,380002,"Revdibazar H.O,Railwaypura,N C Market",23.0333,72.6181,Ahmadabad City
2,380004,"Delhi Gate (Ahmedabad),Cantonment,Girdharnagar...",23.0333,72.6181,Ahmadabad City
3,380005,"Sabarmati,Kabir Chowk,O N G C",23.0833,72.6667,Ahmadabad City
4,380006,Ellisbridge,23.0333,72.6181,Ahmadabad City


In [11]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
def get_number_of_reviews(venue_id):
    tip_url = "https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}".format(venue_id,
        CLIENT_ID, CLIENT_SECRET,VERSION)
    res = requests.get(tip_url).json()
    
    tips = res["response"]
    
    if "tips" in tips.keys():
        return tips["tips"]["count"]
    else:
        return 0

In [13]:
def explore_venues(neigbh , latitude , longitude):
    explore_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET,  VERSION, latitude, longitude,  radius,  LIMIT)

    result = requests.get(explore_url).json()
    
#     print(result.keys())
    venues = result["response"]["groups"][0]["items"]
    
    
#     print(len(venues))
    
    if len(venues)==0:
        return pd.DataFrame()
    
    nearby_venues = json_normalize(venues)

    # # filter columns
    filtered_columns = ['venue.id','venue.name', 'venue.categories','venue.location.city', 'venue.location.lat', 'venue.location.lng', 'venue.location.distance']
    nearby_venues =nearby_venues.loc[:, filtered_columns]

    # filter the category for each row
    nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
    nearby_venues["venue.location.city"] = nearby_venues["venue.location.city"].fillna(method = "ffill")

    # # clean columns
    nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
    
    nearby_venues.loc[:,"place_names"] = neigbh
    nearby_venues.loc[:,"latitude"] = latitude
    nearby_venues.loc[:,"longitude"] = longitude

    return nearby_venues

In [14]:
clusters_df.columns

Index(['postal_code', 'place_names', 'Latitude', 'Longitude',
       'community_name'],
      dtype='object')

In [15]:
all_venues = pd.DataFrame()
for neigbh,lat,long in clusters_df[['place_names', 'Latitude', 'Longitude']].values:
    
    venue_df  = explore_venues(neigbh,lat,long)
    all_venues = pd.concat([all_venues,venue_df],sort=False)

/home/dangar/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [16]:
all_venues.loc[:,"city"] = all_venues.city.fillna("Ahmedabad").values

all_venues.loc[:,"reviews_count"] = all_venues["id"].apply(lambda x : get_number_of_reviews(x))

all_venues_amd = pd.concat([all_venues,pd.get_dummies(all_venues.categories)],axis=1)

all_venues_amd_categories = all_venues_amd.groupby("place_names")[all_venues_amd.columns[11:]].mean()

In [17]:
all_venues_amd_categories = all_venues_amd_categories.reset_index()

In [18]:
all_venues_amd_categories.head()

,place_names,Asian Restaurant,BBQ Joint,Bookstore,Breakfast Spot,Bus Station,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Deli / Bodega,Dessert Shop,Electronics Store,Fast Food Restaurant,Historic Site,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Juice Bar,Lake,Metro Station,Miscellaneous Shop,Multiplex,Park,Pharmacy,Pizza Place,Restaurant,Sandwich Place,Shopping Mall,Tea Room,Tennis Court,Thrift / Vintage Store,Train Station,Vegetarian / Vegan Restaurant,Women's Store
0,"Daxini Society,Khokhara Mehmadabad,L G Hospita...",0.00,0.0,0.0,0.0,0.0,0.0,0.125,0.0,0.0,0.0,0.0,0.125,0.25,0.00,0.0,0.125,0.125,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.125,0.0,0.125,0.00,0.00
1,"Delhi Gate (Ahmedabad),Cantonment,Girdharnagar...",0.00,0.0,0.0,0.5,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.00,0.00,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.50,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000,0.0,0.000,0.00,0.00
2,Ellisbridge,0.00,0.0,0.0,0.5,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.00,0.00,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.50,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000,0.0,0.000,0.00,0.00
3,"Gheekanta Road,Lal Darwaja,Manek Chowk,Distric...",0.05,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.050,0.05,0.05,0.2,0.000,0.350,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.05,0.0,0.0,0.05,0.000,0.0,0.000,0.05,0.05
4,"Gujarat University,Navrangpura H.O",0.00,0.0,0.0,0.5,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.00,0.00,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.50,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000,0.0,0.000,0.00,0.00


### Cluster Areas

In [19]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [20]:
# set number of clusters
kclusters = 6

c_toronto_grouped_clustering = all_venues_amd_categories.drop(['place_names'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(c_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 0, 1, 0, 2, 1, 1, 3, 5], dtype=int32)

In [21]:
all_venues_amd_categories.loc[:,"cluster_no"] = kmeans.labels_
all_venues_amd_updt = all_venues_amd[[ 'name', 'categories', 'city','place_names', 'latitude', 'longitude', 'reviews_count']]

updt_clusters_df = all_venues_amd_updt.merge(all_venues_amd_categories.loc[:,["place_names","cluster_no"]],on="place_names")

print(updt_clusters_df.cluster_no.value_counts())
updt_clusters_df.head()

1    61
0    10
5     5
3     3
4     2
2     1
Name: cluster_no, dtype: int64


,name,categories,city,place_names,latitude,longitude,reviews_count,cluster_no
0,Lucky Tea,Tea Room,Ahmedabad,"Gheekanta Road,Lal Darwaja,Manek Chowk,Distric...",23.0256,72.5769,14,1
1,Hotel Patang,Indian Restaurant,Ahmedabad,"Gheekanta Road,Lal Darwaja,Manek Chowk,Distric...",23.0256,72.5769,10,1
2,Agashiye,Indian Restaurant,Ahmedabad,"Gheekanta Road,Lal Darwaja,Manek Chowk,Distric...",23.0256,72.5769,13,1
3,House of MG,Hotel,Ahmedabad,"Gheekanta Road,Lal Darwaja,Manek Chowk,Distric...",23.0256,72.5769,12,1
4,Hotel Neelkanth Patang,Restaurant,Ahmedabad,"Gheekanta Road,Lal Darwaja,Manek Chowk,Distric...",23.0256,72.5769,2,1


In [22]:
updt_clusters_df

,name,categories,city,place_names,latitude,longitude,reviews_count,cluster_no
0,Lucky Tea,Tea Room,Ahmedabad,"Gheekanta Road,Lal Darwaja,Manek Chowk,Distric...",23.0256,72.5769,14,1
1,Hotel Patang,Indian Restaurant,Ahmedabad,"Gheekanta Road,Lal Darwaja,Manek Chowk,Distric...",23.0256,72.5769,10,1
2,Agashiye,Indian Restaurant,Ahmedabad,"Gheekanta Road,Lal Darwaja,Manek Chowk,Distric...",23.0256,72.5769,13,1
3,House of MG,Hotel,Ahmedabad,"Gheekanta Road,Lal Darwaja,Manek Chowk,Distric...",23.0256,72.5769,12,1
4,Hotel Neelkanth Patang,Restaurant,Ahmedabad,"Gheekanta Road,Lal Darwaja,Manek Chowk,Distric...",23.0256,72.5769,2,1
...,...,...,...,...,...,...,...,...
77,ranip main market,Juice Bar,Ahmedabad,Ranip,23.0801,72.5718,0,5
78,ร้านครัวคุณยาย,Deli / Bodega,Ahmedabad,Ranip,23.0801,72.5718,0,5
79,D k travels,Bus Station,Ahmedabad,Ranip,23.0801,72.5718,0,5
80,E-quality Restaurant,Indian Restaurant,Ahmedabad,Ranip,23.0801,72.5718,0,5


In [81]:
clr_i = 0

final_map = folium.Map(location=(23.0225,72.5714) , zoom_start=11)

colors_array = cm.rainbow(np.linspace(0, 13, updt_clusters_df.shape[0]))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for clstr in updt_clusters_df.cluster_no.unique():
    city_neighbs = updt_clusters_df.loc[updt_clusters_df.cluster_no ==clstr ,["place_names","reviews_count","latitude", "longitude"]]
    neighs = folium.map.FeatureGroup()
    clr = rainbow[clr_i]
    clr_i+=1
    for br,rc,lat,lng in city_neighbs.values:
        folium.CircleMarker(
            [lat, lng],
            radius=3, 
            color=clr,
            fill=True,
            popup= "<br>Neighourhoods ==> {} And <br> revies total ==> {}".format(br,rc),
            fill_opacity=0.8
        ).add_to(final_map)

final_map

In [28]:
all_venues_amd_categories.columns

Index(['place_names', 'Asian Restaurant', 'BBQ Joint', 'Bookstore',
       'Breakfast Spot', 'Bus Station', 'Chinese Restaurant', 'Clothing Store',
       'Coffee Shop', 'Convenience Store', 'Deli / Bodega', 'Dessert Shop',
       'Electronics Store', 'Fast Food Restaurant', 'Historic Site', 'Hotel',
       'Ice Cream Shop', 'Indian Restaurant', 'Indie Movie Theater',
       'Juice Bar', 'Lake', 'Metro Station', 'Miscellaneous Shop', 'Multiplex',
       'Park', 'Pharmacy', 'Pizza Place', 'Restaurant', 'Sandwich Place',
       'Shopping Mall', 'Tea Room', 'Tennis Court', 'Thrift / Vintage Store',
       'Train Station', 'Vegetarian / Vegan Restaurant', 'Women's Store',
       'cluster_no'],
      dtype='object')

In [24]:
Asia_food_category_group = ['Asian Restaurant', 'Chinese Restaurant',"BBQ Joint"]
Cafe_food_category_group = ["Café","Juice Bar","Tea Room","Coffee Shop"]
Fast_food_category_group = ['Fast Food Restaurant','Snack Place','Sandwich Place','Pizza Place',"Ice Cream Shop"]
Veg_food_category_group  = ['Vegetarian / Vegan Restaurant',"Indian Restaurant","Restaurant"]
All_food_category_group = Asia_food_category_group + Cafe_food_category_group + Fast_food_category_group + Veg_food_category_group
Entertainment_category_group = ["Historic Site","Multiplex","Indie Movie Theater"]
Store_category_group = ["Shopping Mall","Electronics Store","Thrift / Vintage Store","Clothing Store","Pharmacy"]
Other_facillitiy_category_group = ["IT Services","Hotel","ATM","Train Station","Massage Studio"] 

In [73]:
def get_recommended_five_places_in_same_clusters(df,sorting_columns):
    
    sorting_columns = list(set(sorting_columns).intersection(df.columns))
    recmd_neighbs =  df.assign(freq = df[sorting_columns].sum(axis=1)).sort_values("freq",ascending=False).reset_index(drop=True)
#     clst_no = recmd_neighbs.loc[0]["cluster_no"]
#     recmd_neighbs = recmd_neighbs.loc[recmd_neighbs.cluster_no == clst_no]
    
    return recmd_neighbs.place_names[:5].tolist()

In [79]:
print("\n".join(get_recommended_five_places_in_same_clusters(all_venues_amd_categories,All_food_category_group)))

Kathwada Maize Product
Limbadia,Naroda,Nana Chiloda,Naroda S A,Medra
Gheekanta Road,Lal Darwaja,Manek Chowk,District Court (Ahmedabad),Gandhi Road (Ahmedabad),Raikhad,Ahmedabad G.P.O.,Raipur (Ahmedabad),Jamalpur (Ahmedabad),Khadia,Dariapur (Ahmedabad),Khanpur (Ahmedabad),Municipal Corporation
Memnagar
Daxini Society,Khokhara Mehmadabad,L G Hospital,Vasisthnagar,S A Mills,Jawahar Chowk,Maninagar


In [18]:
# all_venues.to_csv("last_saved_ahmedabad_data.csv",index=False)

In [2]:
# all_venues = pd.read_csv("last_saved_ahmedabad_data.csv")